In [1]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

import torchvision
from torchvision import models
from torchvision import transforms

from pytorch_metric_learning import distances, losses, miners, reducers, testers
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator

### Load data

In [5]:
images_w_boxes = pd.read_csv('./public_test/images_w_boxes.csv')

In [9]:
submission_list = pd.read_csv('./public_test/submission_list.csv')

In [6]:
class CustomDataset(Dataset):
    
    def __init__(self, df, path):
        self.images = df['img_path'].values
        self.bbox_x1 = df['bbox_x1'].values
        self.bbox_y1 = df['bbox_y1'].values
        self.bbox_x2 = df['bbox_x2'].values
        self.bbox_y2 = df['bbox_y2'].values
        self.path = path
        
    def __getitem__(self, index):
        image_path = self.images[index]
        try:
            image = Image.open(self.path + image_path)
        except:
            image = Image.open(self.images[0])
            print('OPEN FILE ERR', image_path)
        x1 = self.bbox_x1[index]
        y1 = self.bbox_y1[index]
        x2 = self.bbox_x2[index]
        y2 = self.bbox_y2[index]
        try:
            image = image.crop((int(x1), int(y1), int(x2), int(y2)))
        except:
            print('BB ERRR', image_path)
        image = image.resize((224, 224))
        image = transforms.ToTensor()(image)
        try:
            image = transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))(image)
        except:
            image = torch.zeros([3, 224, 224])
            print('IMAGE FORMAT ERRR', image_path)
        if image.shape != torch.Size([3, 224, 224]):
            image = torch.zeros([3, 224, 224])
            print('IMG ERR', image_path)
        return image
        
    def __len__ (self):
        return len(self.images)

In [7]:
dataset = CustomDataset(images_w_boxes, './public_test/images/')

In [8]:
idx2img = {images_w_boxes.loc[i]['img_path'] : images_w_boxes.loc[i].name for i in range(len(images_w_boxes))}

In [10]:
from sklearn.preprocessing import normalize

def get_embedding(model, dataset, img):
    idx = idx2img[img]
    img = dataset[idx]
    emb = normalize(model(torch.unsqueeze(img, 0)).data.cpu().numpy()[0].reshape(1, -1))
    return emb

### Load model

In [2]:
model = models.resnet18(pretrained=False)

In [3]:
model.fc = torch.nn.Identity()

In [4]:
model.load_state_dict(torch.load('./models/model_mc.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

### Prediction

In [11]:
sim_list = []
for i in tqdm(submission_list['id'].values):
    ids, img_a, img_b = submission_list[submission_list['id'] == i].values[0]
    emb_a = get_embedding(model, dataset, img_a)
    emb_b = get_embedding(model, dataset, img_b)
    dist = (2 - np.linalg.norm(emb_a - emb_b)) / 2
    sim_list.append(dist)

  1%|▏         | 188/14960 [00:33<48:16,  5.10it/s]

IMAGE FORMAT ERRR b18bf4af16e58c45f0b44371d73ba6ed.jpg


  7%|▋         | 976/14960 [02:48<40:17,  5.78it/s]

IMAGE FORMAT ERRR b18bf4af16e58c45f0b44371d73ba6ed.jpg


  9%|▊         | 1299/14960 [03:44<39:41,  5.74it/s]

...

 93%|█████████▎| 13934/14960 [37:00<02:40,  6.39it/s]

IMAGE FORMAT ERRR 98eb14af0b772c77c60c09339d96562e.jpg


 94%|█████████▍| 14104/14960 [37:26<02:13,  6.43it/s]

IMAGE FORMAT ERRR 86eae0a5c83f5435019efb3b7cc8a3d2.jpg


100%|██████████| 14960/14960 [39:40<00:00,  6.28it/s]


In [12]:
submission = pd.DataFrame()
submission['id'] = submission_list['id'].values
submission['score'] = sim_list

In [13]:
submission.to_csv('mcs_prediction.csv', index=False)